第一个语法：

In [95]:
SpongeBob  = '''
海绵宝宝= 海绵宝宝 派大星 蟹老板
自己 = 我 | 本宝宝 |派大星我们
寻找 = 看看 | 玩玩 |去捉
活动=  捉水母 | 蟹黄包
'''

In [110]:
Partick = '''
Partick = 寒暄 询问 业务 结尾
寒暄 = 称谓 打招呼 |打招呼
称谓 = 人称 ,
人称 = 海绵宝宝 | 蟹老板 
打招呼 = 你好吗 | 你好
询问 = 你无聊吗想去 | 你想去
业务相关 = 去 业务
业务 = 吃东西 | 打水仗 |捉水母
结尾 = 吗？
'''

In [111]:
def create_grammar(grammar_str, split='=', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [112]:
create_grammar(Partick)

{'Partick': [['寒暄', '询问', '业务', '结尾']],
 '寒暄': [['称谓', '打招呼'], ['打招呼']],
 '称谓': [['人称', ',']],
 '人称': [['海绵宝宝'], ['蟹老板']],
 '打招呼': [['你好吗'], ['你好']],
 '询问': [['你无聊吗想去'], ['你想去']],
 '业务相关': [['去', '业务']],
 '业务': [['吃东西'], ['打水仗'], ['捉水母']],
 '结尾': [['吗？']]}

In [113]:
create_grammar(SpongeBob)

{'海绵宝宝': [['海绵宝宝', '派大星', '蟹老板']],
 '自己': [['我'], ['本宝宝'], ['派大星我们']],
 '寻找': [['看看'], ['玩玩'], ['去捉']],
 '活动': [['捉水母'], ['蟹黄包']]}

In [114]:
generate(gram=create_grammar(Partick,split='='),target='Partick')

'海绵宝宝,你好吗你想去捉水母吗？'

In [115]:
choice = random.choice
def generate(gram, target):
    if target not in gram: return target # means target is a terminal expression
    expaned = [generate(gram, t) for t in choice(gram[target])]
    return ''.join([e if e != '/n' else '\n' for e in expaned if e != 'null'])

In [116]:
for i in range (20):
    print (generate(gram=create_grammar(Partick,split='='),target='Partick'))

你好你想去打水仗吗？
你好你无聊吗想去打水仗吗？
你好吗你无聊吗想去打水仗吗？
你好你想去吃东西吗？
蟹老板,你好吗你无聊吗想去打水仗吗？
你好你想去打水仗吗？
蟹老板,你好吗你无聊吗想去吃东西吗？
你好你无聊吗想去捉水母吗？
蟹老板,你好你想去捉水母吗？
你好吗你无聊吗想去吃东西吗？
你好你想去捉水母吗？
你好吗你无聊吗想去吃东西吗？
海绵宝宝,你好吗你无聊吗想去打水仗吗？
你好你无聊吗想去吃东西吗？
你好你无聊吗想去打水仗吗？
海绵宝宝,你好你无聊吗想去吃东西吗？
你好你想去吃东西吗？
你好吗你想去捉水母吗？
海绵宝宝,你好吗你想去吃东西吗？
海绵宝宝,你好你无聊吗想去捉水母吗？


2. 使用新数据源完成语言模型的训练

In [117]:
filename = 'movie_comments.csv.txt'

In [118]:
import pandas as pd

In [120]:
content = pd.read_csv(filename)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [154]:
content.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [155]:
comment = content['comment'].tolist()

In [156]:
len(comment)

261497

In [180]:
comment[151]

'生活总有这样那样说得出说不出的苦，也许比想象中还要糟一点，但只有咬牙坚持下去，每天给自己一个微笑然后继续努力，总有一天幸福会不期而遇'

In [158]:
import re
import jieba

In [159]:
from collections import Counter

In [142]:
with_jieba_cut =Counter(jieba.cut(articles[11]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\heipi\AppData\Local\Temp\jieba.cache
Loading model cost 0.527 seconds.
Prefix dict has been built succesfully.


In [181]:
def token(string):
    return re.findall('\w+', string)

In [182]:
comment_clean = [''.join(token(str(a))) for a in comment]

In [183]:
with open('movie_commentscl.csv.txt','w',encoding='utf-8') as f:
    for a in comment_clean:
        f.write(a + '\n')

In [198]:
len(comment_clean)

261497

In [184]:
def cut(string): return list(jieba.cut(string))

In [185]:
TOKEN =[]

In [199]:
for i,line in enumerate ((open ('movie_commentscl.csv.txt','rb'))):
    if i%10000==0:
        print (i)
    if i >270000:break
    TOKEN+=cut(line)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000


In [200]:
len(TOKEN)

9503620

In [201]:
words_count = Counter(TOKEN)
words_count.most_common(100)

[('的', 656524),
 ('\r\n', 522994),
 ('了', 204840),
 ('是', 146212),
 ('我', 100676),
 ('都', 72510),
 ('很', 69424),
 ('看', 68044),
 ('电影', 67350),
 ('也', 64130),
 ('和', 62580),
 ('在', 62490),
 ('不', 56870),
 ('有', 55878),
 ('就', 51370),
 ('人', 47818),
 ('好', 45716),
 ('啊', 41606),
 ('这', 34968),
 ('还', 34898),
 ('一个', 34686),
 ('你', 34564),
 ('还是', 32850),
 ('但', 31156),
 ('故事', 30020),
 ('没有', 28686),
 ('就是', 28014),
 ('喜欢', 27132),
 ('让', 26608),
 ('太', 25352),
 ('又', 23132),
 ('剧情', 22718),
 ('没', 21716),
 ('说', 21528),
 ('吧', 21494),
 ('他', 21350),
 ('不错', 20832),
 ('得', 20698),
 ('到', 20682),
 ('给', 20600),
 ('这个', 20116),
 ('上', 20108),
 ('被', 19878),
 ('对', 19648),
 ('最后', 19388),
 ('一部', 19386),
 ('片子', 19180),
 ('什么', 19142),
 ('能', 19064),
 ('与', 18336),
 ('多', 17954),
 ('可以', 17944),
 ('不是', 17622),
 ('最', 17338),
 ('觉得', 17252),
 ('中', 16892),
 ('导演', 16780),
 ('自己', 16708),
 ('拍', 16344),
 ('好看', 16170),
 ('要', 16162),
 ('真的', 15816),
 ('感觉', 15656),
 ('但是', 15446),
 ('里', 15

In [202]:
def prob_1(word):
    return words_count[word]/len(TOKEN)

In [204]:
prob_1('我')

0.010593437027153863

In [205]:
TOKEN[:10]

['吴京', '意淫', '到', '了', '脑残', '的', '地步', '看', '了', '恶心']

In [230]:
TOKEN =[str(t) for t in TOKEN]
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range (len(TOKEN))]

In [231]:
words_count_2=Counter(TOKEN_2_GRAM)

In [238]:
def prob_2(word1, word2):
    if word1+word2 in words_count_2:
        return words_count_2[word1+word2]/len(TOKEN_2_GRAM)
    else:
        return 1/len(TOKEN_2_GRAM)

In [239]:
prob_2('我们', '在')

2.0202827975024253e-05

In [246]:
def get_probablity (sentence):
    words = cut(sentence)
    sentence_pro = 1
    for i, word in enumerate (words[:-1]):
        next_ = words [i+1]
        probablity = prob_2(word, next_)
        sentence_pro*=probablity
    return sentence_pro

In [252]:
get_probablity('战狼真难看')

1.0522306236991799e-07

In [257]:
get_probablity('海绵宝宝')

6.313383742195079e-07

3. 获得最优质的的语言

In [276]:
 setence=[]
for sen in [generate(gram=create_grammar(Partick,split='='), target='Partick') for i in range(20)]:
    setence.append((sen,get_probablity(sen)))

In [277]:
sorted (setence,key=lambda x:x[1],reverse=True)


[('你好你想去捉水母吗？', 8.686862790619655e-43),
 ('你好吗你想去吃东西吗？', 1.790091784935586e-44),
 ('你好你无聊吗想去打水仗吗？', 8.309620956511742e-51),
 ('你好你无聊吗想去打水仗吗？', 8.309620956511742e-51),
 ('你好你无聊吗想去吃东西吗？', 1.4269616631375372e-53),
 ('你好吗你无聊吗想去打水仗吗？', 8.393892136103162e-56),
 ('海绵宝宝,你好你想去打水仗吗？', 7.213501054463657e-57),
 ('你好吗你无聊吗想去吃东西吗？', 1.4414330503661087e-58),
 ('海绵宝宝,你好吗你想去打水仗吗？', 7.286656045049265e-62),
 ('蟹老板,你好吗你想去打水仗吗？', 1.2144426741748774e-62),
 ('蟹老板,你好吗你想去打水仗吗？', 1.2144426741748774e-62),
 ('蟹老板,你好吗你想去打水仗吗？', 1.2144426741748774e-62),
 ('海绵宝宝,你好吗你想去吃东西吗？', 1.2512939980260575e-64),
 ('海绵宝宝,你好你无聊吗想去打水仗吗？', 5.808517147699688e-71),
 ('海绵宝宝,你好你无聊吗想去吃东西吗？', 9.974620181621203e-74),
 ('海绵宝宝,你好吗你无聊吗想去打水仗吗？', 5.867423636247765e-76),
 ('海绵宝宝,你好吗你无聊吗想去捉水母吗？', 4.939106265821037e-82),
 ('海绵宝宝,你好吗你无聊吗想去捉水母吗？', 4.939106265821037e-82),
 ('蟹老板,你好吗你无聊吗想去捉水母吗？', 8.231843776368395e-83),
 ('蟹老板,你好吗你无聊吗想去捉水母吗？', 8.231843776368395e-83)]

# Q&A


0. Can you come up out 3 sceneraies which use AI methods?  
 Siri,  Autopilot,  Smart home
 
 
1. How do we use Github; Why do we use Jupyter and Pycharm;  
See paper and code on Github. Put assignment on Github.  Jupyter is a clear and useful tool to demonstrate the code. Pycharm could be the most powerful tool for python program.
2.What's the Probability Model?
The Probability Model is a mathematical model used to describe the relationship between different random variables. Usually, the probability relationship between one or more random variables is deciphered.


3. Can you came up with some sceneraies at which we could use Probability Model?  
Predict the stock market,  gambling,  Intelligent Transportation System


4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?  
Because so many real world data is unstructured, so coding based on the pattern match could be very complicated .
5.What's the Language Model;
A statistical language model is a probability distribution over sequences of words. Given such a sequence, say of length m, it assigns a probability to the whole sequence.


6. Can you came up with some sceneraies at which we could use Language Model?  
Recognize Speech, translate, Generate speech


7. What's the 1-gram language model  
The model the assigns the probabilities of a word.


8. What's the disadvantages and advantages of 1-gram language model;  
1-gram model just contain one word. But the real word sentence contain so many words, that means the probabilities of one world do not the follow the rules of a sentence. So 1-gram model can not get good result on sentence. 

9. What't the 2-gram models;
The model that assigns these probabilities to sentences or sequence of words is N-Gram. An N-gram is a sequence of N words: a 2-gram is a two-word sequence of words.